In [ ]:
#!/usr/bin/env python
"""
A script to show how to create your own time domain source model.
A simple damped Gaussian signal is defined in the time domain, injected into
noise in two interferometers (LIGO Livingston and Hanford at design
sensitivity), and then recovered.
"""

import numpy as np
import bilby

duration = 4
sampling_frequency = 2048
outdir = 'outdir'
label = 'LIGO_signal'


In [ ]:
injection_parameters = dict(
    mass_1=36.0, mass_2=29.0, a_1=0.4, a_2=0.3, tilt_1=0.0, tilt_2=0.0,
    phi_12=1.7, phi_jl=0.3, luminosity_distance=1000., theta_jn=0.4, psi=0.659,
    phase=1.3, geocent_time=1126259642.413, ra=1.375, dec=-1.2108)

waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                          reference_frequency=20.0 ,
                          minimum_frequency=20.0)

# call the waveform_generator to create our waveform model.
waveform = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    waveform_arguments=waveform_arguments,
    start_time=injection_parameters['geocent_time'] - 3)

In [ ]:

# inject the signal into three interferometers
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
# for interferometer in ifos:
#     interferometer.power_spectral_density = \
#         bilby.gw.detector.PowerSpectralDensity(
#             psd_file="example_aligo.txt")
# ifos.set_strain_data_from_power_spectral_densities(
#     sampling_frequency=sampling_frequency, duration=duration,
#     start_time=injection_parameters['geocent_time'] - 3)

ifos.set_strain_data_from_zero_noise(
    sampling_frequency=sampling_frequency, duration=duration,
    start_time=injection_parameters['geocent_time'] - 3)
ifos.inject_signal(waveform_generator=waveform,
                   parameters=injection_parameters)


In [ ]:
#  create the priors
priors = bilby.gw.prior.BBHPriorDict()

for key in ['a_1', 'a_2', 'tilt_1', 'tilt_2', 'theta_jn', 'phase', 'psi', 'ra',
            'dec', 'phi_12', 'phi_jl', 'luminosity_distance']:
    priors[key] = injection_parameters[key]
    
priors['mass_1'] = bilby.core.prior.Uniform(10, 50, name='mass_1')
priors['mass_2'] = bilby.core.prior.Uniform(10, 50, name='mass_2')
priors['geocent_time'] = bilby.core.prior.Uniform(
    injection_parameters['geocent_time'] - 0.1,
    injection_parameters['geocent_time'] + 0.1, latex_label='$t_c$', unit='s')

# define likelihood
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(ifos, waveform)

# launch sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=1024,
    injection_parameters=injection_parameters, outdir=outdir, label=label)

result.plot_corner()
